# Qual jogador marcará o primeiro gol?

&nbsp;&nbsp;&nbsp;&nbsp;Este *notebook* visa a encontrar um modelo que respondam à pergunta de qual jogador marcará o primeiro gol.

## Importação das bibliotecas necessárias

In [186]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, log_loss, RocCurveDisplay
from sklearn.tree import plot_tree
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import shap

&nbsp;&nbsp;&nbsp;&nbsp;O trecho de código acima importa as bibliotecas que serão utilizadas ao longo do *notebook*. Suas funções estão listadas abaixo:

* **Pandas (`import pandas as pd`)**: Biblioteca para manipulação e análise de dados em Python que permite ler e escrever dados de diferentes formatos (CSV, Excel, SQL, etc.), além de fornecer ferramentas para tratamento de dados, como filtragem, agrupamento, junção, e mais. `pd` é a convenção para importar pandas, facilitando seu uso no código.

* **NumPy (`import numpy as np`)**: Biblioteca de suporte para computação numérica que oferece suporte a arrays e matrizes multidimensionais, além de uma coleção de funções matemáticas. `np` é a convenção para importar o NumPy, facilitando seu uso no código.

* **Matplotlib (`import matplotlib.pyplot as plt`)**: Biblioteca de visualização de dados no Python que permite a criação de gráficos bidimensionais, como gráficos de linha, dispersão, barras, histogramas, etc. `pyplot` é um módulo do Matplotlib que fornece uma interface para criar gráficos de maneira simples e `plt` é a convenção para usá-lo.

* **Seaborn (`import seaborn as sns`)**: Biblioteca de visualização de dados baseada no Matplotlib, mas que fornece uma interface de alto nível para criar gráficos estatísticos. `sns` é a convenção para importar Seaborn.

* **Train-Test Split (`from sklearn.model_selection import train_test_split`)**: Função do Scikit-learn usada para dividir o conjunto de dados em duas partes: dados de treino e dados de teste.

* **Cross-validate (`from sklearn.model_selection import cross_validate`)**: Função que executa a validação cruzada de um modelo, em que o conjunto de dados é dividido em várias partes, garantindo uma avaliação mais robusta.

* **Random Forest Classifier (`from sklearn.ensemble import RandomForestClassifier`)**: Modelo de machine learning que cria múltiplas árvores de decisão e usa a média ou voto majoritário para prever classes.

* **Quadratic Discriminant Analysis (`from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis`)**: Algoritmo de classificação que usa a suposição de que diferentes classes têm distribuições normais com diferentes matrizes de covariância. Ele é útil quando as classes são melhor separadas por fronteiras quadráticas em vez de lineares.

* **GridSearchCV (`from sklearn.model_selection import GridSearchCV`)**: Ferramenta de otimização de hiperparâmetros que permite testar todas as combinações possíveis de parâmetros fornecidos para encontrar a combinação que proporciona o melhor desempenho para o modelo.

* **Confusion Matrix (`from sklearn.metrics import confusion_matrix`)**: Métrica usada para avaliar o desempenho de um modelo de classificação que mostra o número de previsões corretas e incorretas.

* **Accuracy Score (`from sklearn.metrics import accuracy_score`)**: Mede a proporção de previsões corretas em relação ao total de previsões. É uma métrica básica, mas pode ser enganosa em conjuntos de dados desequilibrados.

* **Recall Score (`from sklearn.metrics import recall_score`)**: Mede a proporção de verdadeiros positivos identificados corretamente pelo modelo, ou seja, quantos dos exemplos positivos foram corretamente classificados. É útil quando os falsos negativos são mais importantes que os falsos positivos.

* **ROC AUC Score (`from sklearn.metrics import roc_auc_score`)**: Calcula a Área Sob a Curva ROC (Receiver Operating Characteristic). Essa métrica reflete a capacidade de um modelo em distinguir classes positivas e negativas, sendo uma boa escolha em conjuntos de dados desequilibrados.

* **Log Loss (`from sklearn.metrics import log_loss`)**: Mede o desempenho de um modelo de classificação probabilística, penalizando previsões incorretas com maior severidade do que outras métricas, sendo mais sensível em problemas com classes desequilibradas.

* **RocCurveDisplay (`from sklearn.metrics import RocCurveDisplay`)**: Permite analisar visualmente o desempenho de um classificador em termos de sua capacidade de distinguir entre as classes positivas e negativas.

* **SMOTE (`from imblearn.over_sampling import SMOTE`)**: Técnica de oversampling que gera novos exemplos sintéticos para a classe minoritária, ajudando a balancear conjuntos de dados com classes desequilibradas, melhorando o desempenho do modelo em relação a classes raras.

* **Pipeline (`from imblearn.pipeline import Pipeline`)**: Ferramenta que permite automatizar a execução de atividades em sequência.

* **Shap (`import shap`)**: Biblioteca usada para explicar a saída de modelos de machine learning.

&nbsp;&nbsp;&nbsp;&nbsp;Essas bibliotecas e funções combinadas serão usadas nos modelos de machine learning, para criá-los, otimizá-los e avaliá-los.

## Carregamento dos dados

In [133]:
# Carregar os dados
df = pd.read_csv('players.csv')

&nbsp;&nbsp;&nbsp;&nbsp;Nesse trecho de código, utiliza-se o Pandas para carregar um conjunto de dados (dataset) a partir do arquivo CSV chamado "players.csv". Esse arquivo é resultado do pré-processamento dos dados realizados no *notebook* 'primeiro_jogador_marcar_gol.ipynb'. O nome "df" é escolhido como nome da variável que armazena o DataFrame que contém os dados carregados do arquivo.

## Separação entre features e alvo

In [134]:
# Features que serão usadas para treinar o modelo
features = ['position', 'goals_overall', 'goals_home',
            'goals_away', 'goals_per_90_overall', 'min_per_goal_overall',
            'shots_on_target_per_90_overall', 'shot_accuraccy_percentage_overall',
            'xg_per_90_overall', 'rank_in_club_top_scorer', 'minutes_played_overall',
            'shots_on_target_per_game_overall', 'average_rating_overall']

# Variável alvo
target = 'possible_first_goal'

# Separando as features e o alvo
X = df[features]
y = df[target]

&nbsp;&nbsp;&nbsp;&nbsp;Neste trecho, estão sendo definindas as *features* (variáveis independentes) que serão usadas para treinar o modelo preditivo e também a variável alvo (variável dependente) que o modelo tentará prever. Essas colunas foram selecionadas com base na análise exploratória dos dados realisada no *notebook* 'primeiro_jogador_marcar_gol.ipynb'.

   - **`X = df[features]`**: Seleciona as colunas do `DataFrame` que correspondem às *features*, e armazena essas colunas na variável `X`.
   
   - **`y = df[target]`**: Extraindo a variável alvo do `DataFrame` e a armazena na variável `y`.

&nbsp;&nbsp;&nbsp;&nbsp;Assim, `X` contém as variáveis independentes, e `y` contém a variável que o modelo tentará prever.

## Separação dos dados de teste

In [135]:
# Divide os dados para treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

&nbsp;&nbsp;&nbsp;&nbsp;Esse trecho de código utiliza a função `train_test_split` para dividir o conjunto de dados em partes de treino e teste:

   - **X_train**: Conjunto de dados de entrada (*features*) que será usado para treinar o modelo.

   - **X_test**: Conjunto de dados de entrada que será usado para testar o modelo após o treinamento.

   - **y_train**: Conjunto de dados de saída (*target*) correspondente a **X_train**, usado no treinamento.

   - **y_test**: Conjunto de dados de saída correspondente a `X_test`, usado para avaliar a performance do modelo.

&nbsp;&nbsp;&nbsp;&nbsp;A função `train_test_split(X, y, test_size=0.2, random_state=42)` é responsável pela divisão dos dados. Seus parâmetros são:

   - **`test_size=0.2`**: Esspecifica que 20% dos dados serão reservados para teste, enquanto os 80% restantes serão usados para treinar o modelo.

   - **`random_state=42`**: O parâmetro `random_state` é uma semente de geração de números aleatórios. Quando se define um valor fixo (como 42), a divisão será sempre a mesma em execuções subsequentes, garantindo reprodutibilidade. Se omitido, cada vez que o código for executado, os dados serão divididos de forma diferente.

&nbsp;&nbsp;&nbsp;&nbsp;A divisão dos dados em treino e teste garante que o modelo seja treinado com uma parte dos dados e avaliado com dados que ele não viu durante o treinamento. Isso ajuda a medir o desempenho do modelo e verificar se ele está generalizando bem para novos dados, evitando o *overfitting* (quando o modelo memoriza os dados de treino e falha ao ser testado em novos dados).

## Funções para facilitar a comparação entre os modelos

In [136]:
# Função para realizar a busca de hiperparâmetros
def optimize_model(model, param_grid, X_train, y_train):
    # Cria um objeto GridSearchCV para realizar a busca em grade de hiperparâmetros
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='precision', n_jobs=-1, verbose=1)

    # Treina o modelo utilizando os dados de treinamento
    grid_search.fit(X_train, y_train)

    # Imprime os melhores hiperparâmetros encontrados para o modelo
    print(f"Melhores parâmetros para {model.__class__.__name__}: {grid_search.best_params_}")
    
    # Retorna o modelo ajustado com os melhores hiperparâmetros
    return grid_search.best_estimator_

&nbsp;&nbsp;&nbsp;&nbsp;A função `optimize_model` usa GridSearchCV para realizar a otimização de hiperparâmetros. Os hiperparâmetros são, de acordo com [[1]](#referências), variáveis de configuração externas para gerenciar o treinamento de modelos de machine learning. GridSearchCV é uma função que, segundo [[2]](#referências), realiza a busca exaustiva por todas as combinações de hiperparâmetros fornecidos no `param_grid`, avaliando o modelo para cada conjunto.

&nbsp;&nbsp;&nbsp;&nbsp;Essa função aceita os seguintes parâmetros:

- **model**: O modelo que será otimizado.

- **param_grid**: Um dicionário contendo os hiperparâmetros e seus valores possíveis para o modelo.

- **X_train, y_train**: O conjunto de dados de treino e suas respectivas saídas. Esses dados serão usados para treinar o modelo com diferentes combinações de hiperparâmetros.

&nbsp;&nbsp;&nbsp;&nbsp;Enquanto a função `GridSearchCV` recebe como parâmetros:

- **`estimator=model`**: O modelo que será otimizado, passado como argumento.
- **`param_grid=param_grid`**: O dicionário contendo as possíveis combinações de hiperparâmetros que serão testadas.
- **`cv=5`**: Usa validação cruzada com 5 *folds*. 
- **`scoring='precision'`**: Especifica a métrica que será usada para avaliar o desempenho do modelo durante a otimização, no caso, a precisão.
- **`n_jobs=-1`**: Define o uso de múltiplos núcleos de CPU para acelerar a busca.
- **`verbose=1`**: Faz com que o processo mostre mais detalhes no terminal sobre o progresso da otimização.

&nbsp;&nbsp;&nbsp;&nbsp;Nesse cenário, a validação cruzada, segundo [[3]](#referências) ajuda a garantir que o modelo não seja avaliado apenas em uma única divisão do conjunto de dados, dado que o *dataset* é dividido em K *folds* (5 neste caso). Cada um desses K grupos é separado para teste, enquanto os demais são utilizados para treinamento; após o treinamento, testa-se o modelo com os dados de teste, armazena-se as métricas e descata-se o modelo. Logo, esse método ajuda a evitar a influência da divisão dos dados na métrica.

&nbsp;&nbsp;&nbsp;&nbsp;Dado que a classe positiva é "não marcar o primeiro gol" e o objetivo é  minimizar os falsos positivos, ou seja, os casos em que o modelo prevê que um jogador vai marcar, mas na realidade não marca, nesse contexto, a métrica mais adequada é a Precisão ou *Precision*, a qual mede a proporção de predições positivas corretas em relação ao total de predições positivas, ou seja, ela informa quantos dos jogadores preditos como marcadores realmente marcam. 

&nbsp;&nbsp;&nbsp;&nbsp;Ademais:
- O método `fit` treina o modelo para cada combinação de hiperparâmetros no `param_grid`, utilizando os dados de treino (`X_train` e `y_train`). Ele também executa a validação cruzada para cada combinação de parâmetros. 
- Após a busca, a linha `print(f"Melhores parâmetros para {model.__class__.__name__}: {grid_search.best_params_}")` imprime os melhores hiperparâmetros encontrados para o modelo em questão.
- A linha `return grid_search.best_estimator_` retorna o modelo treinado com os melhores parâmetros encontrados pela GridSearchCV. Esse modelo otimizado pode então ser usado para fazer previsões ou para continuar sendo avaliado em outros dados (como no conjunto de teste).

&nbsp;&nbsp;&nbsp;&nbsp;O processo de otimização de hiperparâmetros deve melhorar o desempenho do modelo, já que, ao testar várias combinações de parâmetros, o modelo é ajustado para maximizar sua performance em termos da métrica escolhida.

In [137]:
# Função para aplicar o SMOTE
def apply_smote (modelo):
  # Cria um objeto SMOTE para lidar com desbalanceamento de classes
  smote = SMOTE(random_state=42)

  # Cria um pipeline que aplica o SMOTE e, em seguida, ajusta o modelo
  pipeline = Pipeline(steps=[('smote', smote), ('model', modelo)])

  # Retorna o pipeline
  return pipeline

&nbsp;&nbsp;&nbsp;&nbsp;O trecho de código acima implementa uma função chamada `apply_smote` que utiliza o método SMOTE (Synthetic Minority Over-sampling Technique) para lidar com o desbalanceamento de classes. Ela cria um pipeline que combina o uso do SMOTE e um modelo *Machine Learning*, aplicando ambos de maneira consecutiva. A função recebe como parâmetro o modelo de aprendizado de máquina que será posteriormente incorporado ao pipeline junto com o SMOTE. Por fim, a função retorna o pipeline, que pode então ser usado diretamente para treinar o modelo com os dados balanceados via SMOTE.

&nbsp;&nbsp;&nbsp;&nbsp;O uso do SMOTE é útil quando as classes estão desbalanceadas, ou seja, a quantidade de exemplos de uma classe é muito menor que a da outra. Isso pode ser problemático porque muitos modelos de aprendizado de máquina têm dificuldade em aprender padrões a partir da classe minoritária. O SMOTE resolve isso gerando exemplos sintéticos, ajudando o modelo a aprender melhor sobre a classe minoritária. No caso da questão de quem marcará o primeiro gol, a classe de quem já marcou é maior que a classe de quem nunca marcou, dessa forma, o SMOTE pode ser uma forma de melhorar a performance do modelo.

In [138]:
# Função para calcular métricas de avaliação dos modelos e gerar visualizações
def metricas(model):
    # Dicionário de métricas que serão calculadas
    scoring = {
        'accuracy': 'accuracy',
        'precision': 'precision_macro',
        'recall': 'recall_macro',
        'f1': 'f1_macro',
        'log_loss': 'neg_log_loss',
        'roc_auc': 'roc_auc'
    }

    # Usar cross_validate para calcular várias métricas
    cv_results = cross_validate(model, X, y, cv=5, scoring=scoring)

    # Mostrar os resultados das métricas calculadas
    print("%0.2f de acurácia com desvio padrão %0.2f" % (np.mean(cv_results['test_accuracy']), np.std(cv_results['test_accuracy'])))
    print("%0.2f de precisão com desvio padrão %0.2f" % (np.mean(cv_results['test_precision']), np.std(cv_results['test_precision'])))
    print("%0.2f de recall com desvio padrão %0.2f" % (np.mean(cv_results['test_recall']), np.std(cv_results['test_recall'])))
    print("%0.2f de f1-score com desvio padrão %0.2f" % (np.mean(cv_results['test_f1']), np.std(cv_results['test_f1'])))
    print("%0.2f de log loss com desvio padrão %0.2f" % (np.mean(-cv_results['test_log_loss']), np.std(cv_results['test_log_loss'])))
    print("%0.2f de ROC AUC com desvio padrão %0.2f" % (np.mean(cv_results['test_roc_auc']), np.std(cv_results['test_roc_auc'])))

    # Treinar o modelo com todo o conjunto de treino
    model.fit(X_train, y_train)

    # Realizar previsões no conjunto de teste
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)

    # Plotar a matriz de confusão normalizada
    cm = confusion_matrix(y_test, y_pred, normalize='true')
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt='.2', cmap='Blues')
    plt.title('Matriz de Confusão Normalizada')
    plt.ylabel('Classe Real')
    plt.xlabel('Classe Predita')
    plt.show()

&nbsp;&nbsp;&nbsp;&nbsp;Essa função, chamada `metricas`, fornece várias métricas de avaliação e gera visualizações que ajudam a interpretar os resultados. A função também utiliza validação cruzada para fornecer estimativas mais confiáveis das métricas.

- O dicionário `scoring` especifica as métricas que serão calculadas:
  - **Acurácia (accuracy)**: Mede a porcentagem de previsões corretas sobre o total de previsões.
  - **Precisão (precision_macro)**: Mede a proporção de predições corretas para cada classe.
  - **Recall (recall_macro)**: Mede quantos dos positivos foram preditos corretamente.
  - **F1-score (f1_macro)**: A média harmônica entre precisão e recall para cada classe.
  - **Log Loss (neg_log_loss)**: Mede a incerteza das previsões, penalizando previsões incorretas.
  - **ROC AUC (roc_auc_score)**: Mede a capacidade do modelo de distinguir entre as classes.
- Os resultados de cada métrica para cada iteração da validação cruzada são armazenados no dicionário `cv_results`.
- Em seguida, as métrica calculadas são impressas. Para cada métrica, a função exibe:
  - A média dos resultados de todas as iterações da validação cruzada.
  - O desvio padrão, que indica a variação nos resultados entre as iterações. Um desvio padrão baixo sugere que o desempenho do modelo é consistente entre as diferentes divisões dos dados.
- **`model.fit(X_train, y_train)`**: O modelo é treinado usando todo o conjunto de treino (`X_train` e `y_train`). Isso garante que o modelo final seja ajustado em todos os dados disponíveis após a validação cruzada.
- A curva ROC (Receiver Operating Characteristic) mostra a relação entre a taxa de verdadeiros positivos e a taxa de falsos positivos, ajudando a avaliar a capacidade do modelo de distinguir entre as classes para diferentes limiares de decisão. A AUC (área sob a curva) mede essa capacidade de forma agregada (ROC AUC).
- A matriz de confusão mostra o desempenho do modelo, comparando as predições com os rótulos reais. Cada célula da matriz indica a quantidade de previsões corretas e incorretas para cada classe. A matriz é normalizada, o que significa que os valores são expressos em porcentagens, facilitando a interpretação visual.

&nbsp;&nbsp;&nbsp;&nbsp;Ao calcular as métricas, a função fornece uma visão do desempenho do modelo, permitindo a avaliação de diferentes aspectos, como a precisão, recall, e capacidade de discriminar entre classes. Ainda, as visualizações ajudam a interpretar o desempenho do modelo, mostrando como ele se comporta em termos de distinção entre as classes (curva ROC) e a distribuição de erros (matriz de confusão). Logo, essa função permite a avaliação e a visualização de modelos de classificação, ajudando a entender seu desempenho.

## Random Forest

&nbsp;&nbsp;&nbsp;&nbsp;Random Forest, de acordo com [[4]](#referências), é um algoritmo de aprendizado de máquina baseado em árvores de decisão, utilizado tanto para classificação quanto para regressão. Ele funciona criando múltiplas árvores de decisão durante o treinamento e combinando suas previsões para melhorar a precisão do modelo. No caso de classificação, a decisão final é feita por meio de votação majoritária entre as árvores. Ao combinar os resultados de várias árvores, o Random Forest tende a ser menos suscetível a overfitting em comparação com uma única árvore de decisão.

### Com hiperparâmetros selecionados manualmente

In [139]:
# Cria o modelo com os hiperparâmetros definidos anteriormente com testes
rf_model_one = RandomForestClassifier(n_estimators=10000, random_state=42, class_weight='balanced')

&nbsp;&nbsp;&nbsp;&nbsp;O trecho de código cria o modelo Random Forest com os mesmos hiperparâmetros utilizados anterioarmente no *notebook* "modelo_primeiro_marcador.ipynb". Tais hiperparâmetros foram selecionados com base em alguns testes manuais:
- **`n_estimators=10000`**: Este hiperparâmetro define o número de árvores de decisão que o modelo vai criar, ou seja, 10.000 árvores; esse número alto pode ajudar a melhorar a precisão, mas também aumenta o tempo de treinamento.
- **`random_state=42`**: Garante que os resultados sejam reproduzíveis. Um valor fixo, o `42` faz com que o processo de aleatoriedade (como a escolha de amostras ou a divisão dos dados) seja sempre o mesmo, permitindo que os resultados sejam replicáveis.
- **`class_weight='balanced'`**: Este hiperparâmetro ajusta os pesos das classes inversamente proporcionais às suas frequências na base de dados. Isso é útil quando os dados são desbalanceados, ou seja, quando uma classe é muito mais frequente do que outra.

### Grid Search

In [ ]:
# Parâmetros para otimização de Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'class_weight': ['balanced', 'balanced_subsample', None]
}

# Aplicar a otimização de hiperparâmetros para cada modelo
optimized_rf = optimize_model(RandomForestClassifier(random_state=42), param_grid_rf, X_train, y_train)

&nbsp;&nbsp;&nbsp;&nbsp;O trecho de código acima faz a otimização dos hiperparâmetros com a função `optimize_model`.

- Parâmetros para otimização (`param_grid_rf`): Um dicionário é definido com os diferentes valores que serão testados para os hiperparâmetros do `RandomForestClassifier`. Cada combinação possível será testada:
     - `n_estimators`: número de árvores no modelo. Os valores testados são 100, 200 e 300.
     - `max_depth`: profundidade máxima das árvores. Os valores testados são 10, 20, 30 e `None` (sem limite de profundidade).
     - `min_samples_split`: número mínimo de amostras necessário para dividir um nó. Os valores testados são 2, 5 e 10.
     - `min_samples_leaf`: número mínimo de amostras necessário em uma folha. Os valores testados são 1, 2 e 4.
     - `bootstrap`: se o modelo usará amostragem com reposição ou não. Os valores testados são `True` (com reposição) e `False` (sem reposição).
     - `class_weight`: ajusta o peso de cada classe para compensar quando uma classe tem muito mais exemplos do que outra.

- Melhores hiperparâmetros: Após testar todas as combinações, os melhores parâmetros encontrados foram:
     - `bootstrap`: True (o modelo usará amostragem com reposição)
     - `max_depth`: 10 (a profundidade máxima das árvores é limitada a 10)
     - `min_samples_leaf`: 1 (cada folha deve conter pelo menos 1 amostra)
     - `min_samples_split`: 2 (cada nó pode ser dividido se houver pelo menos 2 amostras)
     - `n_estimators`: 100 (o modelo usará 100 árvores)
     - `class_weight`: None (o modeo não terá balanceamento de classes)

### Avaliação do modelo

&nbsp;&nbsp;&nbsp;&nbsp;A função `metricas` será aplicada para diferentes possibilidades de aplicação do modelo Random Forest a fim de chegar à melhor versão desse modelo.

In [ ]:
# Aplica a função métricas para obter a avaliação do Random Forest antes do SMOTE e do ajuste de hiperparâmetros
print('Avaliação do Random Foreste antes do SMOTE e do ajuste de hiperparâmetros:')
metricas(rf_model_one)

&nbsp;&nbsp;&nbsp;&nbsp;As métricas obtidas para o modelo Random Forest antes do SMOTE e do ajuste de hiperparâmetros foram:
- **Acurácia (0.73 com desvio padrão de 0.01)**: Mostra que 73% das previsões foram corretas, o que indica que o modelo tem um bom desempenho geral.
- **Precisão (0.58 com desvio padrão de 0.01)**: A precisão de 58% revela que, das vezes que o modelo previu que um jogador marcaria o primeiro gol, apenas 58% das previsões estavam corretas; assim, depreende-se que o modelo prediz muitos falsos positivos (predizendo que o jogador vai marcar, mas ele não marca), o que pode é um ponto de atenção.
- **Recall (0.71 com desvio padrão de 0.03)**: O recall de 71% indica que o modelo é capaz de identificar corretamente 71% dos jogadores que realmente marcaram o primeiro gol; dessa forma, o modelo ainda está perdendo cerca de 29% dos jogadores que realmente marcaram (falsos negativos).
- **F1-Score (0.56 com desvio padrão de 0.01)**: Mostra que o equilíbrio entre precisão e recall não está ideal, indicando que o modelo está errando tanto em prever jogadores incorretos quanto em perder jogadores que realmente marcaram. 
- **Log Loss (0.66 com desvio padrão de 0.10)**: Mostra que o modelo está fornecendo previsões com probabilidades que não são muito precisas.
- **ROC AUC (0.79 com desvio padrão de 0.02)**: Indica que o modelo tem uma boa capacidade de discriminar entre classes (jogadores que marcam ou não o primeiro gol).
- **Matriz de Confusão**:
  - A classe "0" (jogadores que não marcaram o primeiro gol) foi corretamente prevista 72% das vezes, mas houve 28% de falsos positivos (quando o modelo previu que o jogador não marcaria, mas ele marcou).
  - A classe "1" (jogadores que marcaram o primeiro gol) foi corretamente prevista 82% das vezes, com 18% de falsos negativos (o modelo previu que o jogador marcaria o gol, mas ele não marcou).

&nbsp;&nbsp;&nbsp;&nbsp;Portanto, esse modelo tem um desempenho razoável, mas há espaço para melhorias, especialmente no equilíbrio entre precisão e recall. Aplicar SMOTE pode ajudar a melhorar a capacidade do modelo de lidar com classes desbalanceadas, enquanto o ajuste de hiperparâmetros pode aumentar  outras métricas.

In [ ]:
# Aplica a função métricas para obter a avaliação do Random Forest com o SMOTE
print('Avaliação do Random Forest com o SMOTE:')
metricas(apply_smote(rf_model_one))

&nbsp;&nbsp;&nbsp;&nbsp;Análise das métricas e da matriz de confusão para o modelo de Random Forest após a aplicação do SMOTE:
- **Acurácia (0.77 com desvio padrão de 0.01)**: A acurácia aumentou para 77%, indicando que o modelo agora faz mais previsões corretas em relação ao total de amostras, mostrando que o modelo se beneficiou do balanceamento das classes.
- **Precisão (0.57 com desvio padrão de 0.01)**: A precisão diminuiu levemente para 57%, o que indica que, após o SMOTE, o modelo está fazendo mais previsões de falsos positivos (prever que o jogador vai marcar o primeiro gol, mas ele não marca). Isso pode ser um efeito colateral do balanceamento que pode ter feito o modelo superestimar a classe de jogadores que marcam o primeiro gol.
- **Recall (0.67 com desvio padrão de 0.03)**: O recall caiu de 0.71 para 0.67, sugerindo que o modelo agora está capturando um pouco menos de jogadores que realmente marcaram o primeiro gol. Isso pode ter ocorrido porque o SMOTE forçou o modelo a ser mais conservador ao predizer a classe de quem marcará o gol, resultando em mais falsos negativos.
- **F1-Score (0.58 com desvio padrão de 0.02)**: O F1-score teve uma ligeira melhora (de 0.56 para 0.58), indicando que, mesmo com a leve queda na precisão e no recall, o equilíbrio entre essas duas métricas foi mantido.
- **Log Loss (0.63 com desvio padrão de 0.10)**: Houve uma pequena redução no Log Loss (de 0.66 para 0.63), o que sugere que o modelo está mais confiante em suas previsões e comete menos erros severos na atribuição de probabilidades às classes.
- **ROC AUC (0.79 com desvio padrão de 0.02)**: O ROC AUC permaneceu em 0.79, o que é positivo, pois indica que a capacidade do modelo de distinguir entre jogadores que marcam e não marcam o primeiro gol não foi prejudicada pelo SMOTE.
- **Matriz de Confusão**:
  - **Classe 0** (jogadores que não marcaram o primeiro gol) foi corretamente prevista 79% das vezes, um pequeno aumento em relação à versão anterior do modelo. No entanto, houve 21% de falsos positivos, o que é uma leve melhora em relação ao modelo anterior (28%).
  - **Classe 1** (jogadores que marcaram o primeiro gol) foi corretamente prevista 61% das vezes, uma queda em relação aos 82% do modelo anterior. Agora há 39% de falsos negativos, o que significa que o modelo está falhando mais frequentemente em prever corretamente quem vai marcar o primeiro gol.

&nbsp;&nbsp;&nbsp;&nbsp;Dessa forma, o uso do SMOTE ajudou a aumentar a acurácia geral do modelo, mas levou à queda na precisão e no recall. A matriz de confusão revela que o SMOTE pode ter causado um aumento nos falsos negativos.

In [ ]:
# Aplica a função métricas para obter a avaliação do Random Forest com ajuste dos hiperparâmetros
print('Avaliação do Random Forest com ajuste dos hiperparâmetros:')
metricas(optimized_rf)

&nbsp;&nbsp;&nbsp;&nbsp;Análise das métricas e da matriz de confusão para o modelo de Random Forest após o ajuste dos hiperparâmetros, mas sem a aplicação do SMOTE:
- **Acurácia (0.92 com desvio padrão de 0.00)**: A acurácia do modelo é alta, 92%, o que mostra que o modelo está fazendo um bom trabalho geral em prever corretamente a maioria dos casos. Contudo, como sempre, a acurácia deve ser considerada com cuidado, especialmente como as classes são desbalanceadas.
- **Precisão (0.52 com desvio padrão de 0.07)**: A precisão de 52% mostra que o modelo tem dificuldades em prever corretamente os jogadores que marcarão o primeiro gol, com uma quantidade considerável de falsos positivos.
- **Recall (0.50 com desvio padrão de 0.00)**: O recall de 50% indica que o modelo está capturando apenas metade dos jogadores que realmente marcam o primeiro gol. Isso pode ser devido ao desequilíbrio das classes, com o modelo sendo mais tendencioso a prever jogadores que não marcam o gol (classe 0).
- **F1-Score (0.48 com desvio padrão de 0.00)**: O F1-score de 48% reflete que o modelo tem dificuldades para equilibrar a precisão e o recall. Isso significa que o modelo está cometendo tanto falsos positivos quanto falsos negativos.
- **Log Loss (0.23 com desvio padrão de 0.01)**: O log loss de 0.23 indica que o modelo está fazendo boas estimativas de probabilidades para suas previsões.
- **ROC AUC (0.82 com desvio padrão de 0.02)**: Sugere que o modelo é muito bom em discriminar entre as classes (jogadores que marcam e não marcam o primeiro gol).
- **Matriz de Confusão**:
  - **Classe 0** (jogadores que não marcaram o primeiro gol) foi corretamente prevista 99% das vezes, com uma pequena taxa de falsos positivos (0.0096).
  - **Classe 1** (jogadores que marcaram o primeiro gol) foi corretamente prevista apenas 1.9% das vezes, com 98% de falsos negativos. Isso indica que o modelo está extremamente tendencioso para a classe "0" (não marcar o primeiro gol), e quase não consegue identificar a classe de jogadores que marcam o gol.

&nbsp;&nbsp;&nbsp;&nbsp;Esse modelo apresenta uma piora significativa no desempenho de previsão da classe 1 (jogadores que marcam o primeiro gol). A matriz de confusão revela o viés para a classe majoritária (não marcar o gol), o que é uma consequência de não utilizar técnicas para lidar com o desbalanceamento.


In [ ]:
# Aplica a função métricas para obter a avaliação do Random Forest com o SMOTE e com ajuste dos hiperparâmetros
print('Avaliação do Random Forest com SMOTE e com ajuste dos hiperparâmetros:')
metricas(apply_smote(optimized_rf))

&nbsp;&nbsp;&nbsp;&nbsp;Avaliação do Random Forest com SMOTE e ajuste de hiperparâmetros:
- **Acurácia (0.75 com desvio padrão de 0.01)**: A acurácia de 75% indica que o modelo faz previsões corretas em três quartos dos casos, mostrando que, apesar de uma acurácia geral menor em comparação com o modelo sem SMOTE (92%), ele está mais equilibrado em suas previsões.
- **Precisão (0.58 com desvio padrão de 0.01)**: O valor se manteve o mesmo em comparação com o modelo sem o ajuste dos hiperparâmetros, sugerindo que, embora o ajuste de hiperparâmetros tenha melhorado outras métricas, a precisão se manteve estável, o que significa que ainda há falsos positivos consideráveis.
- **Recall (0.71 com desvio padrão de 0.03)**: O recall de 71% significa que o modelo conseguiu identificar corretamente 71% dos jogadores que realmente marcaram o primeiro gol. Esse valor é superior ao recall de 50% do modelo sem SMOTE, mostrando que o ajuste ajudou a recuperar a sensibilidade sem sacrificar tanto a precisão.
- **F1-Score (0.57 com desvio padrão de 0.01)**: Mostra que o modelo ainda tem um equilíbrio entre falsos positivos e falsos negativos. Houve uma ligeira queda em comparação com o modelo sem ajuste, o que sugere que o balanceamento entre as classes ainda precisa de refinamento.
- **Log Loss (0.41 com desvio padrão de 0.03)**: Apresenta uma melhoria significativa em relação aos modelos sem o ajuste dos hiperparâmetros, indicando que o modelo está agora fazendo previsões probabilísticas mais confiantes e com menos erros severos.
- **ROC AUC (0.82 com desvio padrão de 0.02)**: Reflete uma excelente capacidade de discriminação entre os jogadores que marcam e não marcam o primeiro gol, mostrando que o ajuste de hiperparâmetros teve um impacto positivo na capacidade do modelo de diferenciar as duas classes.
- **Matriz de Confusão**:
  - **Classe 0** (jogadores que não marcaram o primeiro gol): O modelo previu corretamente 76% das vezes, com 24% de falsos positivos (o modelo previu que o jogador não faria o gol, mas ele fez).
  - **Classe 1** (jogadores que marcaram o primeiro gol): A previsão correta foi de 74%, com 26% de falsos negativos (o modelo previu que o jogador faria o gol, mas ele não fez).

&nbsp;&nbsp;&nbsp;&nbsp;Dessarte, o ajuste de hiperparâmetros melhorou a capacidade geral do modelo, especialmente em termos de Log Loss e ROC AUC, demonstrando que as previsões probabilísticas são mais confiáveis. Em comparação com o modelo sem SMOTE, a implementação do balanceamento e o ajuste dos hiperparâmetros resultaram em um modelo que não apenas mantém uma boa acurácia, mas também melhora significativamente o recall e a precisão para prever jogadores que marcarão o primeiro gol. No entanto, o equilíbrio entre precisão e recall ainda não é ideal; assim, ainda é possível uma melhoria nesse aspecto.


### Explicabilidade

&nbsp;&nbsp;&nbsp;&nbsp;Segundo [[5]](#referências), a explicabilidade é a capacidade de um sistema de Inteligência Artificial fornecer explicações compreensíveis sobre como chegou a uma determinada decisão ou previsão. Isso envolve a ideia de que um modelo deve ser capaz de justificar seus resultados de forma que seja compreensível para humanos, o que inclui a capacidade de traçar o raciocínio do modelo, identificar quais características foram mais relevantes na tomada de decisão e apresentar isso de uma forma clara e acessível. Logo, a explicabilidade é fundamental em modelos preditivos para aumentar a confiança nas decisões automatizadas, entender os riscos e limitações, identificar possíveis erros, preconceitos ou discriminações embutidos no modelo e para que, em casos de decisões erradas, haja clareza sobre como o modelo operou, permitindo a responsabilização correta. 

In [ ]:
# Importância de cada feature
feature_importances = optimized_rf.feature_importances_
features = X.columns

# Gera uma relação entre as features e sua importância no resultado
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Gráfico para visualização das importâncias das features
plt.figure(figsize=(8, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='skyblue')
plt.xlabel('Importância relativa', fontsize=14)
plt.ylabel('Features', fontsize=14)
plt.title('Features por Importância', fontsize=16)
plt.gca().invert_yaxis()
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp;O gráfico acima, gerado a partir da função `feature_importances_`, mostra a importância relativa de cada uma das *features* usadas no modelo, ou seja, o quanto cada uma delas contribui para a precisão das previsões. Desse modo, a variável "min_per_goal_overall" é a mais importante, isto é, é a variável mais influente nas previsões feitas pelo modelo. Em seguida, temos "goals_per_90_overall" e "goals_overall", que também têm alta importância. As últimas variáveis, como "position" e "average_rating_overall", têm uma importância menor, indicando que elas têm um impacto reduzido nas previsões em comparação com as outras. Dessa forma, esse gráfico é forma de garantir a explicabilidade do modelo, mostrando quais fatores são mais críticos para o resultado previsto.

In [ ]:
# Plota a árvore de decisão a partir do primeiro estimador (primeira árvore) de optmized_fr
plt.figure(figsize=(20,10))
plot_tree(optimized_rf.estimators_[0], filled=True, feature_names=features, class_names=['Marcar', 'Não Marcar'])
plt.savefig('tree_visualization.png', dpi=2000, bbox_inches='tight')


&nbsp;&nbsp;&nbsp;&nbsp;O trecho de código acima está relacionado à visualização de uma das árvores de decisão dentro do modelo Random Forest para entender como ela toma decisões e quais atributos têm maior influência nas previsões. Como um modelo de Random Forest é composto por várias árvores de decisão, o código seleciona a primeira árvore de decisão do modelo (`estimators_[0]`) para visualização. Nessa visualização, cada nó da árvore é colorido de acordo com a classe majoritária em cada nó (neste caso, laranja representa jogadores que marcam o gol e azul, os que não marcam), facilitando a interpretação visual da árvore. Como a árvore é grande, a visualização não é possível no *output* da célula, por isso, uma imagem chamada `tree_visualization.png` é salva.

&nbsp;&nbsp;&nbsp;&nbsp;A visualização de uma única árvore de decisão permite ver como o modelo divide os dados em relação às características dos jogadores, ou seja, essa visualização ajuda a interpretar as regras que a árvore está usando para decidir entre as classes "Marcar" e "Não Marcar". O nó no topo é o nó raiz, onde a primeira decisão é tomada. Neste nó, uma das variáveis do conjunto de dados é usada para dividir os exemplos em dois grupos, de acordo com um critério de decisão (como "X <= valor"). Esse critério é escolhido para maximizar a separação entre as classes (neste caso, "Marcar" e "Não Marcar"). Cada nó intermediário também representa uma divisão com base em uma variável do conjunto de dados. As variáveis de decisão utilizadas nos nós intermediários foram selecionadas pelo algoritmo para maximizar a separação entre os exemplos que marcam gol e aqueles que não marcam. Os nós finais (folhas) representam as previsões finais, isto é, uma folha indica a classe majoritária (neste caso, "Marcar" ou "Não Marcar") após todas as divisões realizadas. 
- Nos nós, há informações como:
     - **Impureza (Gini/Entropia)**: Mostra quão mista em relação às classes é a amostra de exemplos naquele ponto. 
     - **Número de exemplos**: A quantidade de amostras que chegaram a esse ponto na árvore.
     - **Distribuição das classes**: A proporção de amostras de cada classe ("Marcar" ou "Não Marcar") que chegou a esse nó.

&nbsp;&nbsp;&nbsp;&nbsp;O SHAP (SHapley Additive exPlanations) baseia-se na teoria dos valores de Shapley, uma abordagem que atribui a cada característica uma importância para a previsão final do modelo. Essa biblioteca permite interpretar a influência de cada variável nas previsões, tornando os modelos mais transparentes e compreensíveis. Assim, é um método para garantir a explicabilidade de um modelo.

In [ ]:
# Plota os valores SHAP pra o conjunto de teste no optimized_rf
explainer = shap.TreeExplainer(optimized_rf)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values[:, :, 1], X_test)

&nbsp;&nbsp;&nbsp;&nbsp;O trecho de código acima usa a biblioteca SHAP para interpretar o modelo `optimized_rf`. Nesse sentido, o objeto `explainer` usa o modelo treinado `optimized_rf` para calcular os valores SHAP. O `shap_values` armazena os valores SHAP para cada recurso presente no conjunto `X_test`. No código, acessa-se a terceira dimensão (`[:, :, 1]`), que corresponde ao valor SHAP para a predição do jogador que marcará o primeiro gol. A linha `shap.summary_plot(shap_values[:, :, 1], X_test)` cria o gráfico de resumo que mostra:
- **Eixo Y:** As variáveis de entrada (features) que foram usadas no modelo.
- **Eixo X:** Os valores SHAP, que indicam o impacto de cada variável na predição do modelo. Valores à direita de 0 aumentam a chance do jogador marcar o primeiro gol, enquanto valores à esquerda diminuem essa chance.
- **Cores:** A coloração dos pontos (de azul a vermelho) representa o valor da característica. Azul significa valores mais baixos, enquanto vermelho representa valores mais altos.
- **Distribuição:** A dispersão dos pontos mostra a distribuição dos valores SHAP para aquela característica em particular.

&nbsp;&nbsp;&nbsp;&nbsp;Nesse sentido, as variáveis mais importantes estão no topo. Por exemplo, a variável "min_per_goal_overall" tem um grande impacto negativo quando seus valores são altos (azul), e um impacto positivo quando os valores são baixos (vermelho). Além disso, variáveis como "goals_per_90_overall" e "goals_overall" também têm impacto significativo, com valores altos associados a uma maior probabilidade de marcar o primeiro gol. Portanto, o gráfico mostra a importância relativa das características e como seus valores afetam a predição do modelo sobre qual jogador marcará o primeiro gol.

## Quadratic Discriminant Analysis

&nbsp;&nbsp;&nbsp;&nbsp;A *Quadratic Discriminant Analysis* (QDA) ou Análise Discriminante Quadrática, conforme [[6]](#referências), é um algoritmo de classificação que modela a distribuição de dados de cada classe como uma distribuição Gaussiana multivariada com matrizes de covariância distintas. Ou seja, é uma técnica estatística utilizada para classificar dados em diferentes categorias com base em variáveis independentes. É uma extensão da Análise Discriminante Linear (LDA), mas, ao contrário da LDA, a QDA permite que cada classe tenha sua própria matriz de covariância.

### Grid Search

In [ ]:
# Definir os parâmetros a serem otimizados para QDA
qda_param_grid = {
    'reg_param': [0.0, 0.01, 0.1, 0.5, 1.0],
    'tol': [1e-4, 1e-3, 1e-2, 1e-1]
}
# Aplica a função optimize_model no QDA
optimized_qda = optimize_model(QuadraticDiscriminantAnalysis(), qda_param_grid, X_train, y_train)


&nbsp;&nbsp;&nbsp;&nbsp;O trecho de código acima realiza a otimização dos hiperparâmetros do modelo `QuadraticDiscriminantAnalysis` (QDA) utilizando a função `optimize_model`. Um dicionário (`qda_param_grid`) é definido com os diferentes valores que serão testados para os hiperparâmetros do `QuadraticDiscriminantAnalysis` e cada combinação possível será avaliada nos dados de treino:
- **`reg_param`**: parâmetro de regularização que controla o quanto a matriz de covariância é ajustada para evitar overfitting. Os valores testados são 0.0, 0.01, 0.1, 0.5 e 1.0.
- **`tol`**: tolerância para a convergência do modelo, controlando quando o ajuste deve parar. Os valores testados são 1e-4, 1e-3, 1e-2 e 1e-1.

&nbsp;&nbsp;&nbsp;&nbsp;Após o teste de todas as combinações, os melhores parâmetros encontrados foram:
- **`reg_param`:** 0.1 (indica que o modelo aplica uma pequena regularização para evitar que se ajuste demais aos dados)
- **`tol`:** 0.0001 (um valor baixo, indicando que o algoritmo continuará ajustando o modelo até que a mudança seja muito pequena).

&nbsp;&nbsp;&nbsp;&nbsp;Além disso, o código gerou um aviso de colinearidade (`Variables are collinear`), o qual significa que há uma correlação linear entre algumas das variáveis de entrada, o que pode influenciar a performance do modelo. A colinearidade pode causar instabilidade na inversão da matriz de covariância, razão pela qual a regularização foi necessária.

### Avaliações do modelo

In [ ]:
print('Avaliação de Quadratic Discriminant Analysis com ajuste dos hiperparâmetros:')
metricas(optimized_qda)


&nbsp;&nbsp;&nbsp;&nbsp;Avaliação do modelo *Quadratic Discriminant Analysis* com ajuste dos hiperparâmetros:

- **Acurácia (0.80 com desvio padrão de 0.05)**: O modelo apresenta uma acurácia de 80%, o que significa que ele acerta 80% das previsões. Embora essa acurácia seja menor do que a do Random Forest sem SMOTE (92%), ela é comparável ao modelo Random Forest com SMOTE (75%). Isso sugere que o QDA também consegue lidar com a complexidade do problema, mas talvez com menos ajuste necessário.

- **Precisão (0.57 com desvio padrão de 0.01)**: A precisão do QDA é de 57%, o que indica que, embora o modelo consiga fazer boas previsões de jogadores que marcam o primeiro gol, ainda há uma quantidade considerável de falsos positivos, similar ao comportamento do Random Forest com SMOTE, que também apresentou 58% de precisão.

- **Recall (0.65 com desvio padrão de 0.05)**: O recall de 65% é razoavelmente alto e mostra que o modelo captura uma boa quantidade de jogadores que realmente marcaram o primeiro gol. Esse valor está entre o recall do Random Forest sem SMOTE (50%) e o com SMOTE (71%), o que sugere que o QDA também consegue lidar bem com o desbalanceamento de classes.

- **F1-Score (0.58 com desvio padrão de 0.01)**: O F1-Score de 58% reflete um equilíbrio razoável entre precisão e recall. É similar ao F1-Score do Random Forest com SMOTE (0.57), sugerindo que o modelo de QDA tem um desempenho equilibrado ao lidar com falsos positivos e falsos negativos.

- **Log Loss (0.55 com desvio padrão de 0.13)**: O valor de 0.55 para o log loss é superior ao do Random Forest sem SMOTE (0.23), mas ainda está dentro de uma faixa aceitável, indicando que as previsões probabilísticas do QDA são mais incertas ou menos confiáveis em comparação com o Random Forest.

- **ROC AUC (0.82 com desvio padrão de 0.01)**: O ROC AUC de 0.82 mostra que o QDA é muito bom em discriminar entre jogadores que marcam e não marcam o primeiro gol, sendo equivalente ao desempenho dos modelos Random Forest com e sem SMOTE. Isso sugere que, em termos de separação entre classes, o QDA está no mesmo nível de performance.

- **Matriz de Confusão:**
  - **Classe 0** (jogadores que não marcaram o primeiro gol): O modelo previu corretamente 87% das vezes, com 13% de falsos positivos (o modelo previu que o jogador não faria o gol, mas ele fez).
  - **Classe 1** (jogadores que marcaram o primeiro gol): A previsão correta foi de 43%, com 57% de falsos negativos (o modelo previu que o jogador faria o gol, mas ele não fez).

&nbsp;&nbsp;&nbsp;&nbsp;Dessa forma, o QDA demonstra um bom equilíbrio geral de métricas, o que o coloca como uma opção interessante para o problema. Comparando com os modelos de Random Forest, o QDA tem um desempenho geral intermediário, superando o Random Forest sem SMOTE em termos de recall, mas perdendo em precisão e acurácia. Em relação ao Random Forest com SMOTE, o QDA oferece um desempenho muito similar, especialmente em termos de precisão e F1-Score. O maior desafio do QDA é o **log loss**, que é consideravelmente maior (0.55) comparado aos modelos de Random Forest. Isso indica que suas previsões probabilísticas não são tão confiáveis quanto as dos outros modelos.

In [ ]:
print('Avaliação de Quadratic Discriminant Analysis com ajuste dos hiperparâmetros e com SMOTE:')
metricas(apply_smote(optimized_qda))


&nbsp;&nbsp;&nbsp;&nbsp;Avaliação do modelo QDA com ajuste dos hiperparâmetros e aplicação do SMOTE:

- **Acurácia (0.68 com desvio padrão de 0.02)**: A acurácia de 68% é menor em comparação com o QDA sem SMOTE (80%). Isso sugere que o uso de SMOTE pode ter reduzido a acurácia geral ao focar mais no balanceamento das classes.

- **Precisão (0.59 com desvio padrão de 0.00)**: A precisão é de 59%, o que representa uma melhoria em relação ao modelo QDA sem SMOTE (57%). Isso indica que o modelo com SMOTE conseguiu reduzir um pouco a quantidade de falsos positivos em comparação ao modelo sem SMOTE, mas ainda mantém um número considerável de erros.

- **Recall (0.79 com desvio padrão de 0.01)**: O recall aumentou significativamente para 79% com a aplicação de SMOTE (comparado a 65% sem SMOTE), o que indica que o modelo agora é muito mais eficaz em capturar jogadores que realmente marcaram o primeiro gol. Este aumento mostra que o SMOTE foi eficaz em reduzir os falsos negativos.

- **F1-Score (0.56 com desvio padrão de 0.01)**: O F1-score de 56% está levemente abaixo do modelo sem SMOTE (58%). Isso reflete que, embora o recall tenha aumentado significativamente, o equilíbrio entre precisão e recall ainda não é ideal. Ou seja, o modelo está capturando mais jogadores que marcam o primeiro gol, mas em detrimento de mais falsos positivos.

- **Log Loss (1.08 com desvio padrão de 0.21)**: O log loss subiu significativamente (de 0.55 sem SMOTE para 1.08 com SMOTE), o que indica que o modelo está fazendo previsões menos confiáveis após o balanceamento. Isso sugere que o modelo, ao tentar prever com base em dados balanceados artificialmente, se torna mais incerto.

- **ROC AUC (0.82 com desvio padrão de 0.01)**: Assim como nas iterações anteriores, o ROC AUC de 0.82 mostra que o modelo continua sendo muito bom em discriminar entre as classes (jogadores que marcam e não marcam o primeiro gol). Este valor se manteve consistente, o que indica que a capacidade discriminativa do modelo não foi prejudicada pelo SMOTE.

- **Matriz de Confusão**:

	- **Classe 0 (jogadores que não marcaram o primeiro gol)**: O modelo previu corretamente 66% dos jogadores que não marcaram o primeiro gol, com 34% de falsos positivos (onde o modelo previu incorretamente que esses jogadores iriam marcar).

	- **Classe 1 (jogadores que marcaram o primeiro gol)**: O modelo previu corretamente 91% dos jogadores que realmente marcaram o primeiro gol, com apenas 9% de falsos negativos. Este é um grande avanço em relação ao QDA sem SMOTE, onde a previsão correta da classe 1 era de 65%, mostrando o impacto positivo do SMOTE no recall dessa classe.

&nbsp;&nbsp;&nbsp;&nbsp;Dessarte, o QDA com SMOTE mostra um desempenho aprimorado em recall (79%), o que era esperado, já que o SMOTE é projetado para melhorar a detecção da classe minoritária. Isso faz com que o modelo seja capaz de capturar mais jogadores que marcarão o primeiro gol. Porém, a acurácia caiu para 68% e o log loss aumentou significativamente, indicando que o modelo se torna menos confiável em suas previsões. Desse modo, esse modelo é uma boa escolha quando o objetivo é maximizar a detecção da classe minoritária, mesmo que isso implique em sacrificar um pouco a acurácia e a confiança nas previsões. 

## Conclusão
&nbsp;&nbsp;&nbsp;&nbsp;À vista do apresentado, o modelo Random Forest com SMOTE e ajuste de hiperparâmetros é a melhor opção para prever qual jogador marcará o primeiro gol. Esse modelo equilibra bem as métricas de precisão e recall, atingindo 0.58 e 0.71, respectivamente, o que indica que ele é capaz de capturar uma boa proporção de jogadores que realmente marcarão, ao mesmo tempo em que minimiza os falsos positivos. Embora a acurácia de 0.75 seja inferior ao modelo sem SMOTE, o aumento no recall e o bom desempenho em termos de ROC AUC (0.82) mostram que o modelo é eficaz em discriminar entre as classes. Além disso, o log loss de 0.41 aponta que as previsões probabilísticas são confiáveis. Portanto, esse modelo é a escolha mais apropriada para maximizar a detecção da classe minoritária (jogadores que marcam o primeiro gol), sem sacrificar significativamente a acurácia e a confiança nas previsões. Contudo algumas melhorias ainda podem ser realizadas para otimizar ainda mais os resultados, como outras técnicas de balanceamento de classes.

## Referências
[1] O que é o Ajuste de hiperparâmetros? — Métodos de ajuste de hiperparâmetros explicados — AWS. Disponível em: <https://aws.amazon.com/pt/what-is/hyperparameter-tuning/>. Acesso em: 20 set. 2024.

[2] GridSearchCV. Disponível em: <https://scikit-learn.org/dev/modules/generated/sklearn.model_selection.GridSearchCV.html>. Acesso em: 20 set. 2024.

[3] SAVIETTO, J. V. Machine Learning: Métricas, Validação Cruzada, Bias e Variância. Disponível em: <https://medium.com/@jvsavietto6/machine-learning-m%C3%A9tricas-valida%C3%A7%C3%A3o-cruzada-bias-e-vari%C3%A2ncia-380513d97c95>. Acesso em: 20 set. 2024.

[4] EQUIPE EBAC. O que é Random Forest? Disponível em: <https://ebaconline.com.br/blog/random-forest-seo>. Acesso em: 20 set. 2024.

[5] GONÇALVES, B. D. et al. O problema da explicação em Inteligência Artificial: considerações a partir da semiótica. Teccogs: Revista Digital de Tecnologias Cognitivas. v. 17, p. 59–75, 2018.

[6] 1.2. Linear and Quadratic Discriminant Analysis — scikit-learn 0.23.2 documentation. Disponível em: <https://scikit-learn.org/stable/modules/lda_qda.html>. Acesso em: 20 set. 2024.

‌